Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [1]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 4')[0]

Loading in\data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 4']


In [2]:
observations = tab.excel_ref('B24').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B38').expand(DOWN).expand(RIGHT)  


In [3]:
observations

{<E32 10.8>, <J35 23.9>, <M27 1058.0>, <B30 '*'>, <I35 20.1>, <M28 1208.0>, <J36 0.2>, <C27 7.0>, <L30 9.0>, <H24 257.0>, <E23 713.0>, <B33 15.8>, <B31 41.1>, <I28 1057.0>, <L24 385.0>, <K34 12.7>, <H34 20.2>, <I37 '*'>, <D31 43.5>, <L37 0.7>, <M30 158.0>, <B37 '*'>, <K24 649.0>, <D27 26.0>, <B29 '-'>, <G34 14.7>, <F24 425.0>, <G32 28.0>, <H37 '*'>, <I27 1022.0>, <M34 17.7>, <C24 155.0>, <J23 2577.0>, <M26 841.0>, <J31 18.0>, <H26 124.0>, <G25 479.0>, <M24 1498.0>, <B26 15.0>, <G24 494.0>, <J27 559.0>, <C31 47.8>, <B28 28.0>, <D33 12.9>, <F28 588.0>, <I34 19.4>, <F32 17.5>, <E29 '-'>, <M35 20.2>, <K37 0.4>, <J24 464.0>, <G30 '*'>, <E35 21.2>, <K33 16.2>, <G35 12.9>, <E36 '-'>, <I23 5256.0>, <L34 17.7>, <H27 214.0>, <F37 '*'>, <H31 24.2>, <H33 11.7>, <K36 0.0>, <K35 14.5>, <H30 '*'>, <C34 2.2>, <D23 294.0>, <E34 5.0>, <L35 21.9>, <F33 13.5>, <E24 322.0>, <K26 329.0>, <D25 52.0>, <H28 249.0>, <L23 1356.0>, <F35 23.7>, <B27 3.0>, <D34 8.8>, <F29 '*'>, <B34 3.2>, <J37 5.4>, <M29 183.0>, <D

In [4]:
Service = tab.excel_ref('A23').expand(DOWN).is_not_blank()
Service

{<A26 'South Eastern'>, <A23 'Total'>, <A37 'Emergency admissions (HIS) (%)'>, <A32 'Northern (%)'>, <A24 'Belfast'>, <A28 'Western'>, <A33 'South Eastern (%)'>, <A29 'Prison'>, <A30 'Emergency admissions (HIS)'>, <A36 'Prison (%)'>, <A25 'Northern'>, <A35 'Western (%)'>, <A27 'Southern'>, <A34 'Southern (%)'>, <A31 'Belfast (%)'>}

In [5]:
Treatment = tab.excel_ref('B22').expand(RIGHT)
Treatment

{<H22 'Drugs & Alcohol'>, <K22 'Drugs Only'>, <I22 'Total'>, <D22 'Drugs & Alcohol'>, <G22 'Drugs Only'>, <E22 'Total'>, <L22 'Drugs & Alcohol'>, <F22 'Alcohol Only'>, <J22 'Alcohol Only'>, <C22 'Drugs Only'>, <B22 'Alcohol Only'>}

In [6]:
age = tab.excel_ref('B21').expand(RIGHT).is_not_blank()
age

{<B21 'Under 18 '>, <M21 'Overall Total'>, <F21 '18 and over'>, <J21 'Treatment Type'>}

In [7]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Health and Social Care Trust',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Period','1 March 2017'),
            HDimConst('Sex','Persons')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table.loc[new_table['Age'] == 'Treatment Type', 'Age'] = 'All Ages'
new_table.loc[new_table['Age'] == 'Overall Total', 'Age'] = 'All Ages'
new_table.loc[new_table['Treatment Type'] == '', 'Treatment Type'] = 'Total'
new_table

,OBS,DATAMARKER,Treatment Type,Health and Social Care Trust,Age,Measure Type,Unit,Period,Sex
0,95,NaN,Alcohol Only,Total,Under 18,Count,People,1 March 2017,Persons
1,324,NaN,Drugs Only,Total,Under 18,Count,People,1 March 2017,Persons
2,294,NaN,Drugs & Alcohol,Total,Under 18,Count,People,1 March 2017,Persons
3,713,NaN,Total,Total,Under 18,Count,People,1 March 2017,Persons
4,2482,NaN,Alcohol Only,Total,18 and over,Count,People,1 March 2017,Persons
5,1712,NaN,Drugs Only,Total,18 and over,Count,People,1 March 2017,Persons
6,1062,NaN,Drugs & Alcohol,Total,18 and over,Count,People,1 March 2017,Persons
7,5256,NaN,Total,Total,18 and over,Count,People,1 March 2017,Persons
8,2577,NaN,Alcohol Only,Total,Treatment Type,Count,People,1 March 2017,Persons
9,2036,NaN,Drugs Only,Total,Treatment Type,Count,People,1 March 2017,Persons


In [10]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [11]:
new_table = new_table[new_table['Value'] !=  0 ]

In [12]:
new_table = new_table[new_table['Value'] !=  '' ]

In [13]:
new_table.dtypes

Value                           object
DATAMARKER                      object
Treatment Type                  object
Health and Social Care Trust    object
Age                             object
Measure Type                    object
Unit                            object
Period                          object
Sex                             object
dtype: object

In [14]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Health and Social Care Trust,Age,Measure Type,Unit,Period,Sex
161,3.1,NaN,None,Prison (%),Overall Total,Count,People,1 March 2017,Persons
168,5.4,NaN,Alcohol Only,Emergency admissions (HIS) (%),Treatment Type,Count,People,1 March 2017,Persons
169,0.4,NaN,Drugs Only,Emergency admissions (HIS) (%),Treatment Type,Count,People,1 March 2017,Persons
170,0.7,NaN,Drugs & Alcohol,Emergency admissions (HIS) (%),Treatment Type,Count,People,1 March 2017,Persons
171,2.6,NaN,None,Emergency admissions (HIS) (%),Overall Total,Count,People,1 March 2017,Persons


In [15]:
new_table.count()

Value                           146
DATAMARKER                        0
Treatment Type                  131
Health and Social Care Trust    146
Age                             146
Measure Type                    146
Unit                            146
Period                          146
Sex                             146
dtype: int64

In [16]:
new_table = new_table[new_table['Value'] !=  '-' ]

In [17]:
new_table.count()

Value                           146
DATAMARKER                        0
Treatment Type                  131
Health and Social Care Trust    146
Age                             146
Measure Type                    146
Unit                            146
Period                          146
Sex                             146
dtype: int64

In [18]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [19]:
new_table['Treatment Type'].fillna('Total', inplace = True)

In [20]:
new_table.drop(new_table[new_table['Health and Social Care Trust'].isin(['Prison', 'Prison (%)'])].index, inplace = True)

In [21]:
new_table['Service Type'] = 'All'
new_table['Residential Status'] = 'All'
# new_table['Health and Social Care Trust']  = 'All'

In [22]:
new_table = new_table[['Period', 'Sex', 'Age', 'Service Type', 'Residential Status', 'Treatment Type', 'Health and Social Care Trust', 'Measure Type', 'Unit', 'Value']]

In [23]:
new_table.head(5)

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,All,All,Alcohol Only,Total,Count,People,95
1,1 March 2017,Persons,Under 18,All,All,Drugs Only,Total,Count,People,324
2,1 March 2017,Persons,Under 18,All,All,Drugs & Alcohol,Total,Count,People,294
3,1 March 2017,Persons,Under 18,All,All,Total,Total,Count,People,713
4,1 March 2017,Persons,18 and over,All,All,Alcohol Only,Total,Count,People,2482


In [24]:
new_table.count()

Period                          140
Sex                             140
Age                             140
Service Type                    140
Residential Status              140
Treatment Type                  140
Health and Social Care Trust    140
Measure Type                    140
Unit                            140
Value                           140
dtype: int64